In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
import networkx as nx
from nltk.tokenize import sent_tokenize

In [2]:
##INITIALIZING DATA AND TOKENIZING SENTENCES
stories_path = "./redditannotated/story/" ## training data
stories = os.listdir(stories_path)
    
stories_data = []
for story in stories:
    if ".story" in story:
        with open(stories_path+story) as file:
            stories_data.append({"story_id": story, "text": file.read()})
data = pd.DataFrame(stories_data)
data.to_csv('./redditannotated/stories.csv', index=False)    
data = pd.read_csv('./redditannotated/stories.csv')

df = data

#tokenizing sentences
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

#importing stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [11]:
def top3(index):
       
    #from nltk.tokenize import sent_tokenize
    sentences = []
    sentences.append(sent_tokenize(df['text'].iloc[index]))
    sentences = [y for x in sentences for y in x] # flatten list
        
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences.astype(str)]
    
    
    #function to remove stop words
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new
    
    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in str(clean_sentences)]
                  
    #extract word vectors -- word embeddings removed from function to above (save time)
   
    #REMOVING FIRST TWO SENTENCES
    #clean_sentences = clean_sentences[2:]
    
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            # original_plus_emotion_vector = np.concat([word_embeddings.get(w, np.zeros((100,))), 
            #                                            sentiment_for_this_word])
            v = sum([word_embeddings.get(w, np.zeros((100,))) 
                     for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    
    from sklearn.metrics.pairwise import cosine_similarity
    
    #for i in range(len(sentences)):
    #    for j in range(i, len(sentences)):
            #if i != j:
    #        sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100),
    #                                        sentence_vectors[j].reshape(1,100))[0,0]
    
    for i in range(len(sentences)):
        for j in range(i, len(sentences)):
          #if i != j:
            _dist = cosine_similarity(sentence_vectors[i].reshape(1,100),
                                            sentence_vectors[j].reshape(1,100))[0,0]
            sim_mat[i][j] = _dist
            sim_mat[j][i] = _dist   
    
    # applying page rank algorithm
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    
    length = len(sentences)
    
    #number the sentences REMOVE COMMENT BELOW FOR FULL SENTENCES********
    for i in range(length):
        sentences[i] = str(i) #+ " " + sentences[i]
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    top3 = ""
    j = 0
    i=0
    
    top3 = []
    length = len(ranked_sentences)
    for i in range(length):
        if int(ranked_sentences[i][1]) != 0 and int(ranked_sentences[i][1]) != 1 and j < 3:
            top3.append(int(ranked_sentences[i][1]))
            j+=1
         
    return top3 #int()   [int(x) for x in top3]

In [74]:
def checkIndex(storyid):
    for i in range(5000):
        if df['story_id'].iloc[i] == storyid:
            return i
            break

In [13]:
def annotations(index):
    story_id = df['story_id'].iloc[index]
    #print(story_id)
    story1 = story_id.split('.')[-3]
    #print(story1)
    story2 = story_id.split('.')[-2]
    #print(story2)
    story_id = story1 + "." +story2+"."
    #print(story_id)
    
    sentences = []
    
    sentences.append(sent_tokenize(df['text'].iloc[index]))
    sentences = [y for x in sentences for y in x] # flatten list
    
    annotatedstr = ""
    
    annotationsarray = []
    #./reddit annotated/human_annotations'
    fh = open('./redditannotated/union')
    i=0
    for line in fh:
        if line.startswith(story_id):
    i        #line.split('.')[-1]
            gtindex = int(line.split('.')[-1])
            #annotatedstr += str(gtindex) + " "
            #print(gtindex)
            annotationsarray.append(int(gtindex))

            
    fh.close()
    annotatedstr = annotatedstr.replace('\n', '')
    return annotationsarray

In [14]:
def linegraph(index):
    gt = annotations(index)
    pred = top3(index)
    storyname = df['story_id'].iloc[index]
    filepath = "./redditannotated/story/" + storyname
    
    count = (len(open(filepath).readlines()))    
    
    linegraph = ""
    for i in range(count):
        #print(i, type(i), gt, type(gt), pred, type(pred))
        if i in gt and i in pred and i!=0 and i!=1:
            linegraph+="\u25C9"
        elif i in gt:
            linegraph+="\u25D2"
        elif i in pred and i!=0 and i!=1:
            linegraph+="\u25D3"
        else:
            linegraph+="\u25EF"
            
    return linegraph

In [15]:
def visualizeGraph(index):
    # tokenizing sentences   
    from nltk.tokenize import sent_tokenize
    sentences = []
    sentences.append(sent_tokenize(df['text'].iloc[index]))
    sentences = [y for x in sentences for y in x] # flatten list
        
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences.astype(str)]
    
    #importing stopwords moved to above
    
    #function to remove stop words
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new
    
    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in str(clean_sentences)]
                  
    #extract word vectors -- word embeddings removed from function to above (save time)
   
    #REMOVING FIRST TWO SENTENCES
    #clean_sentences = clean_sentences[2:]
    
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            # original_plus_emotion_vector = np.concat([word_embeddings.get(w, np.zeros((100,))), 
            #                                            sentiment_for_this_word])
            v = sum([word_embeddings.get(w, np.zeros((100,))) 
                     for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    
    from sklearn.metrics.pairwise import cosine_similarity
    
            
    for i in range(len(sentences)):
        for j in range(i, len(sentences)):
          #if i != j:
            _dist = cosine_similarity(sentence_vectors[i].reshape(1,100),
                                            sentence_vectors[j].reshape(1,100))[0,0]
            sim_mat[i][j] = _dist
            sim_mat[j][i] = _dist
            
    
    # applying page rank algorithm
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    
    length = len(sentences)
    
    #number the sentences REMOVE COMMENT BELOW FOR FULL SENTENCES********
    for i in range(length):
        sentences[i] = str(i) #+ " " + sentences[i]
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    plt.subplot(121)
    graph = nx.draw(nx_graph, with_labels=True)
    
    return graph
    
    

In [54]:
def printInfo(index):
    sentences = []
    sentences.append(sent_tokenize(df['text'].iloc[index]))
    sentences = [y for x in sentences for y in x]
    three_random_numbers = [None,None,None]
    
    for i in range(3):
        three_random_numbers[i] = random.randint(1, len(sentences))
        
    #a, b, c, d = str(index), df['story_id'][index], annotations(index), top3(index)
    a, b, c, d, e = (str(index), df['story_id'].iloc[index], list(set(annotations(index))), 
                        list(set(top3(index))), str(linegraph(index)))
     
    #UNCOMMENT LINE FOR RANDOM GENERATION               
    #a, b, c, d, e = (str(index), df['story_id'].iloc[index], list(set(annotations(index))), three_random_numbers, str(linegraph(index)))

    print(a + " | " + b + "| " , c , "| " , d , " | " + e)
    return [a, b, c, d, e]
    

In [79]:
printInfo(180)

180 | 1h7qys.44.story|  [0] |  [3, 4, 5]  | ◒◯◯◓◓◓


['180', '1h7qys.44.story', [0], [3, 4, 5], '◒◯◯◓◓◓']

In [63]:
def get_pres_and_recall(results):
    def get_set(x):
        return set(x)
    
    gt_sets = results.GT.apply(get_set)
    predicted_sets = results.PREDICTED.apply(get_set)
    
    local_presc = 0
    total_pred = 0
    total_gt = 0
    for gt, pred in zip(gt_sets, predicted_sets):
        for element in pred:
            if element in gt:
                local_presc += 1
            total_pred += 1
        total_gt += len(gt)
    
    #print("local_presc, total_pred, total_gt")
    #print(local_presc, total_pred, total_gt)
    print(f"pres:{local_presc/total_pred} recall:{local_presc/total_gt}")
    #return (local_presc, total_pred, total_gt, local_presc/total_pred, local_presc/total_gt)


In [55]:
results = []
print("INDEX | STORY | GT | PREDICTED | LINE GRAPH")
print("For line graph: GT = /  PRED = \\  BOTH = |")
for i in range(470):
    results.append(printInfo(i))

INDEX | STORY | GT | PREDICTED | LINE GRAPH
For line graph: GT = /  PRED = \  BOTH = |
0 | hzqqn.49.story|  [32, 29, 30, 31] |  [17, 36, 31]  | ◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◓◯◯◯◯◯◯◯◯◯◯◯◒◒◉◒◯◯◯◓◯◯
1 | 1mg1dm.16.story|  [40, 41, 20] |  [16, 19, 23]  | ◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◓◯◯◓◒◯◯◓◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◒◒◯◯◯◯
2 | 12he9h.15.story|  [2] |  [2]  | ◯◯◉
3 | vfxbo.63.story|  [5] |  [2, 3, 5]  | ◯◯◓◓◯◉◯
4 | vfxbo.26.story|  [8] |  [9, 2, 3]  | ◯◯◓◓◯◯◯◯◒◓◯◯◯◯◯
5 | bhbyx.21.story|  [4, 5] |  [3, 4, 5]  | ◯◯◯◓◉◉◯
6 | vfxbo.3.story|  [11] |  [24, 7, 23]  | ◯◯◯◯◯◯◯◓◯◯◯◒◯◯◯◯◯◯◯◯◯◯◯◓◓◯◯◯◯◯◯◯◯◯◯◯◯
7 | hzqqn.7.story|  [4, 5, 6] |  [3, 4, 5]  | ◯◯◯◓◉◉
8 | pwlhs.2.story|  [7] |  [3, 5, 6]  | ◯◯◯◓◯◓◓◒◯
9 | hzqqn.89.story|  [20] |  [16, 8, 12]  | ◯◯◯◯◯◯◯◯◓◯◯◯◓◯◯◯◓◯◯◯◒◯◯◯
10 | hzqqn.31.story|  [18, 20, 22] |  [2, 5, 6]  | ◯◯◓◯◯◓◓◯
11 | hzqqn.74.story|  [8, 7] |  [9, 3, 4]  | ◯◯◯◓◓◯◯◒◒◓
12 | 1r034d.6.story|  [6, 7] |  [8, 5, 6]  | ◯◯◯◯◯◓◉◒◓
13 | 27fdf1.13.story|  [6, 7] |  [3, 5, 6]  | ◯◯◯◓◯◓◉◒
14 | vfxbo.87.story|  [9] |  [2, 6, 7]

123 | 1cy98r.17.story|  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19] |  [8, 2, 21]  | ◯◯◓◯◯◯◯◯◓◯◒◒◒◒◒◒◒◒◒◒◯◓◯◯◯◯◯
124 | 12he9h.32.story|  [5] |  [2, 3, 5]  | ◯◯◓◓◯◉◯
125 | uie6f.42.story|  [6] |  [2, 4, 5]  | ◯◯◓◯◓◓◒◯◯◯◯◯
126 | hzqqn.4.story|  [4, 5, 7] |  [2, 6, 7]  | ◯◯◓◯◒◒◓◉
127 | 1h7qys.23.story|  [11] |  [11, 3, 7]  | ◯◯◯◓◯◯◯◓◯◯◯◉◯◯◯◯
128 | 1h7qys.6.story|  [3, 4, 5, 6, 7] |  [9, 3, 4]  | ◯◯◯◉◉◒◒◒◯◓
129 | vfxbo.25.story|  [11] |  [8, 2, 7]  | ◯◯◓◯◯◯◯◓◓◯◯◒◯
130 | vfxbo.60.story|  [7] |  [8, 2, 4]  | ◯◯◓◯◓◯◯◒◓
131 | bhbyx.22.story|  [16, 13, 14, 15] |  [9, 5, 13]  | ◯◯◯◯◯◓◯◯◯◓◯◯◯◉◒◒◒◯
132 | pwlhs.19.story|  [18, 19, 20, 21, 22] |  [11, 3, 15]  | ◯◯◯◓◯◯◯◯◯◯◯◓◯◯◯◓◯◯◒◒◒◒◒◯◯◯◯◯◯◯◯◯◯
133 | 1mg1dm.15.story|  [2, 13, 18, 19, 20] |  [16, 17, 20]  | ◯◯◒◯◯◯◯◯◯◯◯◯◯◒◯◯◓◓◒◒◉
134 | xd222.16.story|  [3, 4] |  [8, 9, 7]  | ◯◯◯◒◒◯◯◓◓◓◯
135 | xd222.5.story|  [2, 3] |  [2, 3, 4]  | ◯◯◉◉◓
136 | hzqqn.77.story|  [1] |  [4, 6, 7]  | ◯◒◯◯◓◯◓◓
137 | hzqqn.32.story|  [] |  [2, 19, 20]  | ◯◯◓◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◯◓◓◯◯

249 | 11urgx.12.story|  [6] |  [5, 6, 7]  | ◯◯◯◯◯◓◉◓
250 | vfxbo.133.story|  [8, 7] |  [4, 6, 7]  | ◯◯◯◯◓◯◓◉◒
251 | 1r034d.11.story|  [1, 2, 3, 4, 5] |  [4, 13, 15]  | ◯◒◒◒◉◒◯◯◯◯◯◯◯◓◯◓◯
252 | vfxbo.71.story|  [16] |  [4, 12, 7]  | ◯◯◯◯◓◯◯◓◯◯◯◯◓◯◯◯◒◯◯◯◯◯
253 | 28jz2y.6.story|  [0] |  [3, 4, 5]  | ◒◯◯◓◓◓◯
254 | vfxbo.34.story|  [4, 5] |  [4, 5, 6]  | ◯◯◯◯◉◉◓◯◯◯
255 | uie6f.16.story|  [5] |  [17, 10, 4]  | ◯◯◯◯◓◒◯◯◯◯◓◯◯◯◯◯◯◓◯◯◯
256 | 12he9h.42.story|  [9, 10] |  [3, 12, 5]  | ◯◯◯◓◯◓◯◯◯◒◒◯◓◯
257 | uie6f.32.story|  [2, 3] |  [8, 3, 4]  | ◯◯◒◉◓◯◯◯◓
258 | 11urgx.9.story|  [6, 7] |  [8, 3, 7]  | ◯◯◯◓◯◯◒◉◓
259 | vfxbo.55.story|  [3] |  [2, 3]  | ◯◯◓◉
260 | vfxbo.10.story|  [13] |  [11, 12, 5]  | ◯◯◯◯◯◓◯◯◯◯◯◓◓◒◯
261 | 1h7qys.53.story|  [8, 5, 6, 7] |  [9, 3, 5]  | ◯◯◯◓◯◉◒◒◒◓◯
262 | hzqqn.82.story|  [5, 6] |  [2, 4, 5]  | ◯◯◓◯◓◉◒
263 | hzqqn.42.story|  [4] |  [2, 3]  | ◯◯◓◓
264 | oeo0h.2.story|  [15] |  [8, 12, 5]  | ◯◯◯◯◯◓◯◯◓◯◯◯◓◯◯◒◯◯◯◯◯◯◯◯◯◯◯
265 | vfxbo.68.story|  [18] |  [9, 13, 7]  | ◯◯◯◯◯◯◯

375 | 27fdf1.5.story|  [8, 9] |  [2, 3, 5]  | ◯◯◓◓◯◓◯◯◒
376 | hzqqn.65.story|  [1] |  [2, 3]  | ◯◒◓◓
377 | hzqqn.20.story|  [1] |  [4, 14, 15]  | ◯◒◯◯◓◯◯◯◯◯◯◯◯◯◓◓◯◯◯◯◯◯◯◯◯◯◯◯◯◯
378 | oeo0h.14.story|  [6, 7] |  [9, 3, 7]  | ◯◯◯◓◯◯◒◉◯◓◯
379 | y9dex.19.story|  [2, 3, 4, 7] |  [2, 6, 7]  | ◯◯◉◒◒◯◓◉
380 | 1mg1dm.6.story|  [7, 8, 10, 11, 12, 13, 14] |  [12, 13, 6]  | ◯◯◯◯◯◯◓◒◒◯◒◒◉◉◒◯
381 | hzqqn.98.story|  [6] |  [3, 4, 5]  | ◯◯◯◓◓◓◒◯◯◯
382 | 1mg1dm.42.story|  [2, 7] |  [5, 6, 7]  | ◯◯◒◯◯◓◓◉
383 | y9dex.24.story|  [3] |  [2, 3]  | ◯◯◓◉
384 | 1pqs6b.11.story|  [11, 12] |  [11, 5, 6]  | ◯◯◯◯◯◓◓◯◯◯◯◉◒
385 | uie6f.15.story|  [0, 1] |  [4, 5, 6]  | ◒◒◯◯◓◓◓
386 | 28jz2y.5.story|  [2] |  [3, 4, 5]  | ◯◯◒◓◓◓
387 | vfxbo.37.story|  [] |  [2, 4, 7]  | ◯◯◓◯◓◯◯◓
388 | vfxbo.72.story|  [7] |  [9, 3, 4]  | ◯◯◯◓◓◯◯◒◯◓
389 | bhbyx.77.story|  [37, 38, 39, 40, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] |  [31, 37, 15]  | ◯◯◯◯◯◯◯◯◯◯◯◒◒◒◒◉◒◒◒◒◒◒◯◯◯◯◯◯◯◯◯◓◯◯◯◯◯◉◒◒◒◯◯◯◯◯◯◯
390 | 28jz2y.7.story|  [5, 6, 7] |  [9,

In [69]:
results_df = pd.DataFrame(results, columns=["INDEX", "STORY", "GT", "PREDICTED", "LINE GRAPH"])

In [70]:
random_results_df = pd.DataFrame(results, columns=["INDEX", "STORY", "GT", "PREDICTED", "LINE GRAPH"])

In [71]:
selected_results40 = results_df[results_df['LINE GRAPH'].apply(len) <= 40]
selected_results20 = results_df[results_df['LINE GRAPH'].apply(len) <= 20]

In [64]:
get_pres_and_recall(selected_results40)

pres:0.20015396458814472 recall:0.3051643192488263


In [65]:
get_pres_and_recall(selected_results20)

pres:0.22128851540616246 recall:0.3640552995391705


In [66]:
get_pres_and_recall(results_df)

pres:0.19420289855072465 recall:0.27714581178903824


In [67]:
get_pres_and_recall(random_results_df)

pres:0.17768595041322313 recall:0.24457083764219234


In [172]:
results_df.columns

Index(['INDEX', 'STORY', 'GT', 'PREDICTED', 'LINE GRAPH'], dtype='object')